In [ ]:
import json
import sys
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil

In [ ]:
main_folder = '13zebra'
exps = os.listdir(main_folder)
outdir = os.path.join('.','ap36k_zebra')
outdir_images = os.path.join(outdir,'images')
outdir_labels = os.path.join(outdir,'labels')
outdir_labels_coco = os.path.join(outdir,'labels_coco')
for od in [outdir, outdir_images, outdir_labels, outdir_labels_coco]:
    if not os.path.exists(od):
        os.makedirs(od)
        
for exp in exps:
    files = os.listdir(os.path.join(main_folder,exp))
    for file in files:
        if file[-5:] == '.json':
            fname = file[:-5]
            out_fname = f'{exp}_{fname}'
            shutil.copy(os.path.join(main_folder,exp,f"{fname}.jpg"),os.path.join(outdir_images,out_fname+".jpg"))
            with open(os.path.join(main_folder,exp,f"{fname}.json"),"r") as f:
                annos = json.load(f)
                imwidth = annos['imageWidth']
                imheight = annos['imageHeight']
                
                with open(os.path.join(outdir_labels,out_fname+".txt"),'w') as of, open(os.path.join(outdir_labels_coco,out_fname+".txt"),'w') as of_coco:
                    for shape in annos['shapes']:
                        if shape['shape_type'] == 'rectangle': # is bbox
                            c = 0
                            x_center = (shape['points'][0][0] + shape['points'][1][0])/(2*imwidth)
                            y_center = (shape['points'][0][1] + shape['points'][1][1])/(2*imheight)
                            lx = min(shape['points'][0][0], shape['points'][1][0])
                            rx = max(shape['points'][0][0], shape['points'][1][0])
                            uy = min(shape['points'][0][1], shape['points'][1][1])
                            dy = max(shape['points'][0][1], shape['points'][1][1])
                            width = (rx-lx)/(imwidth)
                            height = (dy-uy)/(imheight)
                            of.write(f"{c} {x_center} {y_center} {width} {height}\n")
                            c = 22
                            of_coco.write(f"{c} {x_center} {y_center} {width} {height}\n")


# create train and val jsons and copy the images to the target folders

In [ ]:
myl = ['video2','video5','video13','video19']

In [ ]:
def map_keypoints(k):
    print("WRONG MAPPING FUNCTION!!!! Check neck and root of tail")
    tmp = [0]*81
    tmp[0:3] = k[39:42] # left back paw
    tmp[3:6] = k[36: 39] # left back knee
    tmp[6:9] = k[33:36] # left back thigh
    
    tmp[9:12]  = k[48:51] # r back paw
    tmp[12:15] = k[45:48] # r back knee
    tmp[15:18] = k[42:45] # r back thigh
    
    tmp[18:21] = k[30:33] # r f p
    tmp[21:24] = k[27:30] # r f k
    tmp[24:27] = k[24:27] # r f t
    
    tmp[27:30] = k[21:24] # l f p
    tmp[30:33] = k[18:21] # l f k
    tmp[33:36] = k[15:18] # l f t
    
    #tmp[36:39] tail end
    #tmp[39:42] tail start
    #tmp[42:45, 45:48, 48:51, 51:54] # right ear tip, base, left ear tip, base
    
    tmp[54:57] = k[3:6] # right eye
    tmp[57:60] = k[0:3] # left eye
    tmp[60:63] = k[6:9] # nose
    
    tmp[78:81] = k[9:12] # neck # wrong
    # tmp[63:66] = k[9:12] # neck # correct  
    tmp[75:78] = k[12:15] # root of tail # wrong
    # tmp[39:42] = k[12:15] # root of tail # correct
    
    # 63-66, neck start 
    # 66-69, neck end 
    # 69-72, skull 
    # 72-75, body middle 
    # 75-78, back end 
    # 78-81 back front
    return tmp

In [ ]:
f = json.load(open("/ps/project/irotate/vit_data/apt36k/train/train.json",'r'))
train = []
val = []
train_ann = []
val_ann = []
train_id = []
val_id = []
train_targets = ['/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/real_zebras_old/train/images']
valid_targets = ['/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/real_zebras_old/valid/images']
sf = '/media/ebonetto/WindowsData/real_zebras/ap36k_zebra/valid/images'
for j in f['images']:
    if '13zebra' in j['file_name']:
        tmpfn = 'video'+j['file_name'].split('video')[1].split('_')[0]
        j['file_name'] = 'video'+j['file_name'].split('video')[1].replace('\\','_')
        if tmpfn not in myl:
            train.append(j)
            train_id.append(j['id'])
            #for tar in train_targets:
            #    shutil.copy(f"/media/ebonetto/WindowsData/real_zebras/ap36k_zebra/valid/images/{j['file_name']}", tar)
        else:
            val.append(j)
            val_id.append(j['id'])
            #for tar in valid_targets:
            #    shutil.copy(f"/media/ebonetto/WindowsData/real_zebras/ap36k_zebra/valid/images/{j['file_name']}", tar)
                
for j in f['annotations']:
    j['category_id'] = 23
    j['num_keypoints'] = 27
    if j['image_id'] in train_id:
        j['keypoints'] = map_keypoints(j['keypoints'])
        train_ann.append(j)
    elif j['image_id'] in val_id:
        j['keypoints'] = map_keypoints(j['keypoints'])
        val_ann.append(j)
    

In [ ]:
f2 = json.load(open("/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/apt36k_annotations.json",'r'))

In [ ]:
f['images'] = val
f['annotations'] = val_ann
f['categories'] = [{'name': 'person', 'id': 1, 'supercategory': 'person'},
 {'supercategory': 'animal',
  'id': 23,
  'name': 'zebra',
  'keypoints': ['left_back_paw',
   'left_back_knee',
   'left_back_thigh',
   'right_back_paw',
   'right_back_knee',
   'right_back_thigh',
   'right_front_paw',
   'right_front_knee',
   'right_front_thigh',
   'left_front_paw',
   'left_front_knee',
   'left_front_thigh',
   'tail_end',
   'tail_base',
   'right_ear_tip',
   'right_ear_base',
   'left_ear_tip',
   'left_ear_base',
   'right_eye',
   'left_eye',
   'nose',
   'neck_start',
   'neck_end',
   'skull',
   'body_middle',
   'back_end',
   'back_front'],
  'skeleton': [[1, 2],
   [2, 3],
   [3, 26],
   [4, 5],
   [5, 6],
   [6, 26],
   [7, 8],
   [8, 9],
   [10, 11],
   [11, 12],
   [13, 14],
   [15, 16],
   [17, 18],
   [16, 19],
   [19, 20],
   [18, 20],
   [19, 21],
   [20, 21],
   [19, 24],
   [20, 24],
   [21, 24],
   [24, 23],
   [23, 22],
   [22, 27],
   [27, 9],
   [27, 12],
   [27, 25],
   [25, 26],
   [26, 14]]},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'}]


In [ ]:
json.dump(f, open("/media/ebonetto/WindowsData/pose_zebras_sw/ViTPose/data/real_zebras_full_kps_valid.json",'w'))

# Map json

In [ ]:
cats = [{'name': 'person', 'id': 1, 'supercategory': 'person'},
 {'supercategory': 'animal',
  'id': 23,
  'name': 'zebra',
  'keypoints': ['left_back_paw',
   'left_back_knee',
   'left_back_thigh',
   'right_back_paw',
   'right_back_knee',
   'right_back_thigh',
   'right_front_paw',
   'right_front_knee',
   'right_front_thigh',
   'left_front_paw',
   'left_front_knee',
   'left_front_thigh',
   'tail_end',
   'tail_base',
   'right_ear_tip',
   'right_ear_base',
   'left_ear_tip',
   'left_ear_base',
   'right_eye',
   'left_eye',
   'nose',
   'neck_start',
   'neck_end',
   'skull',
   'body_middle',
   'back_end',
   'back_front'],
  'skeleton': [[1, 2],
   [2, 3],
   [3, 26],
   [4, 5],
   [5, 6],
   [6, 26],
   [7, 8],
   [8, 9],
   [10, 11],
   [11, 12],
   [13, 14],
   [15, 16],
   [17, 18],
   [16, 19],
   [19, 20],
   [18, 20],
   [19, 21],
   [20, 21],
   [19, 24],
   [20, 24],
   [21, 24],
   [24, 23],
   [23, 22],
   [22, 27],
   [27, 9],
   [27, 12],
   [27, 25],
   [25, 26],
   [26, 14]]},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'}]

In [ ]:
folder = '/ps/project/irotate/vit_data/ap-10k/annotations'
file = 'ap10k-test-split1.json'
out_file = f'{file[:-5]}_27.json'

f = json.load(open(os.path.join(folder, file),'r'))
for j in f['annotations']:
    # j['category_id'] = 23
    j['num_keypoints'] = 27
    j['keypoints'] = map_keypoints(j['keypoints'])
    
f['categories'] = cats
json.dump(f, open(os.path.join(folder, out_file),'w'))

# merge jsons

In [93]:
import pycocotools.coco as COCO
thefile = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27_train_red.json'
of      = "/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/syn_zebras/train/s_UDA_27_train_red.json"
sf = "/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/syn_zebras/train/train_gt30.json"
j1 = json.load(open(thefile,'r'))

cats = j1['categories']
j2 = json.load(open(sf,'r'))
cats += j2['categories']
# skeleton = j2['categories'][1]['skeleton']
# keypoints = j2['categories'][1]['keypoints']

j1 = COCO.COCO(thefile)
j2      = COCO.COCO(sf)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=5.31s)
creating index...
index created!


In [94]:
cats.pop(1)

{'name': 'person', 'id': 1, 'supercategory': 'person'}

In [95]:
import random
from copy import deepcopy

tmp = {}
tmp['images'] = []
tmp['annotations'] = []
tmp['categories'] = []
counter = 1
ann_counter = 1
for j in j1.getImgIds():
    im = j1.loadImgs(j)[0]
    newid = counter
    im['id'] = counter
    im['file_name'] = "../../real_horse_crop/"+im['file_name']
    tmp['images'].append(im)
    for ann in j1.loadAnns(j1.getAnnIds(imgIds=j)):
        ann['id'] = ann_counter
        ann['image_id'] = counter
        tmp['annotations'].append(deepcopy(ann))
        ann_counter += 1
    counter += 1
print(len(tmp['annotations']))
del j

for j in j2.getImgIds():
    im = j2.loadImgs(j)[0]
    newid = counter
    im['id'] = counter
    tmp['images'].append(im)
    for ann in j2.loadAnns(j2.getAnnIds(imgIds=j)):
        ann['id'] = ann_counter
        ann['image_id'] = counter
        # ann['category_id'] = 22
        tmp['annotations'].append(deepcopy(ann))
        ann_counter += 1
    counter += 1
print(len(tmp['annotations']))

80
240677


In [96]:
tmp['categories'] = cats
# for cat in tmp['categories']:
#     cat['skeleton'] = skeleton
#     cat['keypoints'] = keypoints
json.dump(tmp, open(of,'w'))
del j1
del j2 
del tmp

In [ ]:
tmp = json.load(open(of))
tmp['images'][0]

# FIX DATA 

In [ ]:

def fixann(ann):
    ann[78:81], ann[63:66] = ann[63:66], ann[78:81] 
    ann[75:78], ann[39:42] = ann[39:42], ann[75:78]
    return ann

In [ ]:
fname = '/ps/project/irotate/vit_data/ap-10k/annotations/ap10k-test-split1_27.json'
jfile = json.load(open(fname))

for ann in jfile['annotations']:
    ann['keypoints'] = fixann(ann['keypoints'])
    
json.dump(jfile, open(fname[:-5]+'_fixed.json','w'))

# CAT FIXER

In [ ]:
import json
import os

fi = '/ps/project/irotate/vit_data/ap-10k/annotations/ap10k-val-split1_OZ_27_wrong_tail_neck.json'
ff = '/ps/project/irotate/vit_data/ap-10k/annotations/ap10k-val-split1.json'

jsonfile = json.load(open(fi,'r'))
jsonfile_fixed = json.load(open(ff,'r'))

cats = jsonfile['categories']
cats_fixed = jsonfile_fixed['categories']

In [ ]:
for cat in cats:
    print(cat['id'], cat['name'])

In [ ]:
fname_mapping = {}
for i in jsonfile['images']:
    fname_mapping[i['id']] = i['file_name']  
    
fname_OZ = set()
for i in jsonfile_fixed['images']:
    fname_OZ.add(i['file_name'])

In [ ]:
cnt = set()
for ann in jsonfile['annotations']:
    if ann['category_id'] == 23:
        if fname_mapping[ann['image_id']] in fname_OZ:
            cnt.add(fname_mapping[ann['image_id']])
            ann['category_id'] = 22
      

In [ ]:
cnt = set()
for ann in jsonfile['annotations']:
    if ann['category_id'] == 22:
        cnt.add(fname_mapping[ann['image_id']])
len(cnt)

In [ ]:
fname_fixed_mapping = {}
for i in jsonfile_fixed['images']:
    fname_fixed_mapping[i['id']] = i['file_name']
cnt_fixed = set()
for ann in jsonfile_fixed['annotations']:
    if ann['category_id'] == 22:
        cnt_fixed.add(fname_fixed_mapping[ann['image_id']])

In [ ]:
len(cnt_fixed)

In [ ]:
cats = cats_fixed

In [ ]:
json.dump(jsonfile, open(fi,'w'))

# Add 1s to json

In [ ]:
import json
import os

fi = '/ps/project/irotate/vit_data/syn_zebras/valid/valid_gt30.json'
ff = '/ps/project/irotate/vit_data/syn_zebras/valid/valid_gt30_1s.json'

jsonfile = json.load(open(fi,'r'))

img_width = 1920
img_height = 1080

for i, ann in enumerate(jsonfile['annotations']):
    for j in range(0, len(ann['keypoints']), 3):
        if ann['keypoints'][j+2] == 0 and ann['keypoints'][j] > 0 and ann['keypoints'][j+1] > 0 and ann['keypoints'][j] < img_width and ann['keypoints'][j+1] < img_height:
            ann['keypoints'][j+2] = 1
    jsonfile['annotations'][i] = ann
json.dump(jsonfile, open(ff,'w'))

# Take out zebras from json

In [ ]:
import json
import os

fi = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/ap-10k/annotations/ap10k-test-split1_27.json'
ff = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/ap-10k/annotations/ap10k-test-split1_27_OZ.json'

jsonfile = json.load(open(fi,'r'))


In [ ]:
for cat in jsonfile['categories']:
    print(cat['id'], cat['name'])

In [ ]:
toremove = []
for i, ann in enumerate(jsonfile['annotations']):
    if ann['category_id'] != 22:
        toremove.append(i)

for i in sorted(toremove, reverse=True):
    del jsonfile['annotations'][i]
    
json.dump(jsonfile, open(ff,'w'))

# Convert tigdog

In [ ]:
import json
import os
fi = '/ps/project/irotate/vit_data/unsup_dom_adapt/real_horse_crop.json'
ff = '/ps/project/irotate/vit_data/unsup_dom_adapt/real_horse_crop_17.json'
ff_27 = '/ps/project/irotate/vit_data/unsup_dom_adapt/real_horse_crop_27.json'

jsonfile = json.load(open(fi,'r'))

In [ ]:
def keep_17(ann):
    kps = [0] * (17 * 3)
    for i in range(len(ann)):
        if i % 3 == 0:
            if i == 0:
                kps[0:3] = ann[i:i+3]
            elif i == 3:
                kps[3:6] = ann[i:i+3]
            elif i == 6:
                kps[6:9] = ann[i:i+3] # note that this might be misleading
            elif i == 9:
                kps[21:24] = ann[i:i+3]
            elif i == 12:
                kps[30:33] = ann[i:i+3]
            elif i == 15:
                kps[39:42] = ann[i:i+3]
            elif i == 18:
                kps[48:51] = ann[i:i+3]
            elif i == 21:
                kps[12:15] = ann[i:i+3]
            elif i == 24:
                kps[18:21] = ann[i:i+3]
            elif i == 27:
                kps[27:30] = ann[i:i+3]
            elif i == 30:
                kps[36:39] = ann[i:i+3]
            elif i == 33:
                kps[45:48] = ann[i:i+3]
            elif i == 36 or i == 39 or i == 54:
                continue
            elif i == 42:
                kps[15:18] = ann[i:i+3]
            elif i == 45:
                kps[24:27] = ann[i:i+3]
            elif i == 48:
                kps[33:36] = ann[i:i+3]
            elif i == 51:
                kps[42:45] = ann[i:i+3]
    return kps    

In [ ]:
jsonfile['categories'][0]['keypoints'] = ['left_back_paw',   'left_back_knee',
   'left_back_thigh','right_back_paw','right_back_knee','right_back_thigh','right_front_paw','right_front_knee','right_front_thigh','left_front_paw','left_front_knee','left_front_thigh','tail_end','tail_base','right_ear_tip','right_ear_base','left_ear_tip','left_ear_base','right_eye','left_eye','nose','neck_start','neck_end','skull','body_middle','back_end','back_front'],
jsonfile['categories'][0]['skeleton']=  [[1, 2],
   [2, 3],   [3, 26],   [4, 5],   [5, 6],   [6, 26],   [7, 8],   [8, 9],   [10, 11],   [11, 12],   [13, 14],   [15, 16],   [17, 18],   [16, 19],   [19, 20],   [18, 20],   [19, 21],   [20, 21],   [19, 24],   [20, 24],   [21, 24],   [24, 23],   [23, 22],   [22, 27],   [27, 9],   [27, 12],   [27, 25],   [25, 26],   [26, 14]]

In [ ]:
for ann in jsonfile['annotations']:
    ann['keypoints'] = keep_17(ann['keypoints'])

json.dump(jsonfile, open(ff,'w'))

In [ ]:
def map_tigdog_27(k):
    # print("WRONG MAPPING FUNCTION!!!! Check neck and root of tail")
    tmp = [0]*81
    tmp[0:3] = k[39:42] # left back paw
    tmp[3:6] = k[36:39] # left back knee
    tmp[6:9] = k[33:36] # left back thigh
    
    tmp[9:12]  = k[48:51] # r back paw
    tmp[12:15] = k[45:48] # r back knee
    tmp[15:18] = k[42:45] # r back thigh
    
    tmp[18:21] = k[30:33] # r f p
    tmp[21:24] = k[27:30] # r f k
    tmp[24:27] = k[24:27] # r f t
    
    tmp[27:30] = k[21:24] # l f p
    tmp[30:33] = k[18:21] # l f k
    tmp[33:36] = k[15:18] # l f t
    
    #tmp[36:39] tail end
    #tmp[39:42] tail start
    #tmp[42:45, 45:48, 48:51, 51:54] # right ear tip, base, left ear tip, base
    
    tmp[54:57] = k[3:6] # right eye
    tmp[57:60] = k[0:3] # left eye
    tmp[60:63] = k[6:9] # nose
    
    # tmp[78:81] = k[9:12] # neck # wrong
    tmp[63:66] = k[9:12] # neck # correct  
    # tmp[75:78] = k[12:15] # root of tail # wrong
    tmp[39:42] = k[12:15] # root of tail # correct
    
    # 63-66, neck start 
    # 66-69, neck end 
    # 69-72, skull 
    # 72-75, body middle 
    # 75-78, back end 
    # 78-81 back front
    return tmp

In [ ]:
for ann in jsonfile['annotations']:
    ann['keypoints'] = map_tigdog_27(ann['keypoints'])
    

In [ ]:
json.dump(jsonfile, open(ff_27,'w'))

# reduce number of syn images

In [ ]:
import json
import os
ff = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/syn_zebras/train/train_gt30.json'
fo = ff[:-5]+'_3k_bis.json'

jsonfile = json.load(open(ff,'r'))
m = {}
for i in jsonfile['images']:
    m[i['id']] = i['file_name']

ids = []
allids = []
import random

for idx, ann in enumerate(jsonfile['annotations']):
    if not ('_' in m[ann['image_id']] or  'video' in m[ann['image_id']] or 'clip' in m[ann['image_id']] or 'frame' in m[ann['image_id']] or '(' in m[ann['image_id']]):
        allids.append(idx)
    # keep only bboxes if the longest dimension is greater than 100 px
    if ann['bbox'][2] > 100 or ann['bbox'][3] > 100:
        ids.append(idx)
      
ids = random.sample(ids, k=360 if 'val' in ff else 2640)
import numpy as np
ids = np.array(ids)
allids = np.array(allids)
toremove = np.setdiff1d(allids, ids)
toremove.sort()
for i in toremove[::-1]:
    jsonfile['annotations'].pop(i)

print(len(jsonfile['annotations']))
json.dump(jsonfile, open(fo,'w'))

In [ ]:
ff = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/syn_zebras/train/train_gt30_3k_bis.json'
jsonfile = json.load(open(ff,'r'))
n_kps = []
for ann in jsonfile['annotations']:
    n_kps.append(sum(ann['keypoints'][2::3])/2) 

In [ ]:
ff = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/syn_zebras/train/train_gt30_3k_bis.json'
fi = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/s_apt36k/train/train_full.json'
fo = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/s_apt36k/train/train_full_3k_bis.json'

jsonfile = json.load(open(ff,'r'))
jsonfile2 = json.load(open(fi,'r'))

m = {}
for i in jsonfile['images']:
    m[i['id']] = i['file_name']
    
bbox_as_keys = {}
for ann in jsonfile['annotations']:
    if m[ann['image_id']] not in bbox_as_keys:
        bbox_as_keys[m[ann['image_id']]] = []
    bbox_as_keys[m[ann['image_id']]].append(ann['bbox'])
    
m_new = {}
for i in jsonfile2['images']:
    m_new[i['id']] = i['file_name']
    
torem = []
for idx, ann in enumerate(jsonfile2['annotations']):
    found = False
    if m_new[ann['image_id']] in bbox_as_keys:
        for bbox in bbox_as_keys[m_new[ann['image_id']]]:
            if ann['bbox'] == bbox:
                found = True
                break
    if m_new[ann['image_id']].startswith('000') or "_" in m_new[ann['image_id']] or "frame" in m_new[ann['image_id']] or "video" in m_new[ann['image_id']] or "clip" in m_new[ann['image_id']] or "v" in m_new[ann['image_id']]:
        found = True
    if not found:
        torem.append(idx)
print(len(jsonfile2['annotations'])-len(torem))
jsonfile3 = json.load(open(fo[:-9]+".json",'r'))
print(len(jsonfile3['annotations']))

In [ ]:
torem.sort()
for i in torem[::-1]:
    jsonfile2['annotations'].pop(i)
json.dump(jsonfile2, open(fo,'w'))

# Check and fix cats

In [ ]:
import json
import os

In [ ]:
folder = '/ps/project/irotate/vit_data/zebra-zoo/annotations'
fnames = os.listdir(folder)
fnames = [f for f in fnames if '27' in f and 'wrong' not in f]

In [ ]:
for fname in fnames:
    print()
    c = set()
    jsonfile = json.load(open(os.path.join(folder,fname),'r'))
    for ann in jsonfile['annotations']:
        c.add(ann['category_id'])
    print(fname, len(c))
    c = set()
    for cat in jsonfile['categories']:
        c.add(cat['id'])
    print(fname, len(c))
    print()
    for cat in jsonfile['categories']:
        cat['skeleton'] = [[1, 2], [2, 3], [3, 26], [4, 5], [5, 6], [6, 26], [7, 8], [8, 9], [10, 11], [11, 12], [13, 14], [15, 16], [17, 18], [16, 19], [19, 20], [18, 20], [19, 21], [20, 21], [19, 24], [20, 24], [21, 24], [24, 23], [23, 22], [22, 27], [27, 9], [27, 12], [27, 25], [25, 26], [26, 14]]
        cat['keypoints'] = ['left_back_paw', 'left_back_knee', 'left_back_thigh', 'right_back_paw', 'right_back_knee', 'right_back_thigh', 'right_front_paw', 'right_front_knee', 'right_front_thigh', 'left_front_paw', 'left_front_knee', 'left_front_thigh', 'tail_end', 'tail_base', 'right_ear_tip', 'right_ear_base', 'left_ear_tip', 'left_ear_base', 'right_eye', 'left_eye', 'nose', 'neck_start', 'neck_end', 'skull', 'body_middle', 'back_end', 'back_front']
    json.dump(jsonfile, open(os.path.join(folder,fname),'w'))


In [ ]:
tmp = '/ps/project/irotate/vit_data/zebra-zoo/annotations/val_27kps_fixed.json'
tmp2 = '/ps/project/irotate/vit_data/apt36k/valid/valid_27_fixed.json'

In [ ]:
j2 = json.load(open(tmp2,'r'))
j1 = json.load(open(tmp,'r'))

In [ ]:
j1['categories'] = j2['categories']
json.dump(j1, open(tmp,'w'))

# 1250 Ap10K (25*50)

In [97]:
# Annotation list from ScarceNet
anno_list = [48752, 48839, 48787, 48876, 48866, 48850, 48859, 48768, 48725, 48848, 48738, 48790, 48864, 48817, 48728, 48852, 48827, 48904, 48753, 48746, 48796, 48808, 48907, 48782, 48751, 49094, 49026, 49000, 48950, 49031, 48951, 49084, 48912, 49071, 48944, 48966, 48971, 49041, 49085, 48949, 49099, 48984, 49025, 49001, 49012, 49051, 49028, 49075, 49055, 49082, 28605, 26976, 29435, 29350, 30228, 29940, 29485, 29380, 28269, 30273, 27713, 30006, 29326, 29873, 29067, 26970, 27099, 29116, 30350, 29091, 29152, 27269, 28943, 29510, 28852, 30999, 30740, 31066, 30595, 30977, 30971, 31643, 31554, 30806, 30970, 30984, 30651, 30507, 31566, 30817, 31443, 31088, 30983, 31133, 30990, 31577, 31233, 30992, 31199, 31555, 49245, 49312, 49309, 49406, 49265, 49287, 49269, 49226, 49323, 49317, 49303, 49276, 49346, 49264, 49291, 49349, 49296, 49410, 49236, 49350, 49263, 49360, 49369, 49241, 49290, 49984, 50041, 50193, 50104, 50220, 50053, 50095, 50208, 50055, 50214, 49978, 50052, 50235, 50105, 50004, 50238, 49993, 50191, 50213, 50036, 50231, 50145, 50022, 49983, 50147, 200, 149, 103, 135, 237, 180, 108, 155, 58, 217, 282, 143, 234, 107, 226, 221, 96, 172, 260, 240, 125, 106, 715, 90, 84, 1079, 1038, 1075, 1092, 1083, 1050, 1072, 1060, 1073, 1091, 1030, 1061, 1098, 1103, 1028, 1108, 1036, 1040, 1025, 1037, 1109, 1064, 1031, 1058, 1049, 1513, 1218, 1935, 1680, 1193, 1977, 1128, 1122, 1197, 1217, 1813, 1182, 1144, 1132, 1236, 1117, 1702, 1126, 1175, 1143, 1114, 1222, 1967, 1224, 1172, 2156, 2109, 2106, 2158, 2151, 2104, 2111, 2015, 2173, 1988, 1983, 2174, 2138, 2069, 2099, 2072, 2160, 2095, 2112, 2123, 2146, 2088, 2075, 2150, 2159, 4521, 4331, 3346, 4238, 5033, 4657, 4086, 4781, 4768, 4422, 2821, 3717, 2688, 5433, 4879, 2455, 5096, 4706, 4109, 2610, 4373, 5444, 4853, 4817, 5011, 7102, 7249, 7753, 7745, 7773, 7286, 6966, 7409, 7274, 7213, 8050, 7446, 8063, 7964, 7336, 8469, 7728, 7557, 7734, 5822, 7763, 6111, 5862, 5989, 8246, 43256, 43101, 43109, 43203, 43179, 43211, 43126, 43193, 43189, 43250, 43087, 43105, 43107, 43113, 43093, 43174, 43213, 43095, 43226, 43188, 43244, 43210, 43092, 43106, 43231, 45128, 45039, 45087, 45098, 45001, 44976, 44943, 44926, 44969, 44960, 44953, 45082, 44962, 45015, 44998, 44935, 44930, 45014, 45086, 45116, 45020, 45013, 45113, 44986, 44981, 46201, 46273, 46157, 46215, 46251, 46286, 46271, 46165, 46183, 46193, 46168, 46256, 46233, 46176, 46238, 46209, 46228, 46290, 46178, 46268, 46284, 46265, 46109, 46149, 46151, 20054, 20033, 20072, 20075, 20040, 20038, 20066, 20020, 20023, 20049, 20055, 20044, 20078, 20043, 20017, 20067, 20039, 20057, 20032, 20051, 20076, 20026, 20010, 20013, 20061, 20165, 20191, 20234, 20103, 20155, 20100, 20180, 20184, 20122, 20172, 20127, 20235, 20158, 20242, 20213, 20230, 20237, 20173, 20104, 20194, 20124, 20116, 20176, 20139, 20121, 20329, 20280, 20250, 20252, 20328, 20278, 20245, 20336, 20277, 20246, 20270, 20279, 20332, 20275, 20330, 20265, 20290, 20253, 20272, 20305, 20298, 20311, 20249, 20304, 20292, 20399, 20455, 20354, 20376, 20568, 20338, 20368, 20427, 20452, 20496, 20535, 20385, 20493, 20403, 20437, 20475, 20566, 20581, 20543, 20339, 20557, 20538, 20411, 20480, 20422, 20682, 20639, 20665, 20610, 20592, 20670, 20655, 20646, 20601, 20632, 20645, 20607, 20634, 20609, 20661, 20642, 20684, 20683, 20630, 20672, 20643, 20599, 20635, 20602, 20620, 48578, 48706, 48607, 48710, 48546, 48649, 48692, 48713, 48696, 48708, 48639, 48567, 48587, 48542, 48584, 48678, 48637, 48707, 48571, 48724, 48547, 48716, 48570, 48588, 48615, 31989, 32064, 31957, 32017, 32013, 32054, 31950, 31930, 32065, 32072, 31904, 31919, 31969, 31952, 31923, 32047, 32073, 32005, 32029, 31954, 32002, 31892, 31932, 32011, 32071, 35695, 34250, 40406, 32828, 40436, 40808, 35616, 40782, 40047, 40658, 39933, 40778, 40269, 34461, 40492, 40428, 40470, 36017, 40413, 34984, 40812, 35550, 33306, 34906, 36239, 36434, 36338, 36481, 36482, 36367, 36319, 36394, 36428, 36397, 36365, 36477, 36420, 36511, 36342, 36361, 36493, 36487, 36470, 36321, 36341, 36333, 36307, 36362, 36404, 36349, 37590, 37591, 37728, 37559, 37826, 37679, 37713, 37833, 37584, 37815, 37522, 37697, 37805, 37626, 37534, 37601, 37618, 37526, 37588, 37698, 37736, 37685, 37785, 37681, 37733, 37888, 37889, 37891, 37892, 37893, 37894, 37895, 37866, 37867, 37870, 37871, 37872, 37874, 37876, 37877, 37878, 37881, 37882, 37883, 37884, 37885, 37886, 38497, 38286, 38041, 37993, 38674, 38353, 37979, 38675, 38031, 38486, 37909, 38035, 37953, 38036, 37901, 37968, 37910, 38408, 37948, 38704, 38032, 38017, 37997, 38047, 38023, 38752, 38832, 38785, 38739, 38855, 38840, 38748, 38914, 38728, 38836, 38737, 38819, 38721, 38863, 38792, 38811, 38860, 38825, 38852, 38738, 38900, 38843, 38731, 38745, 38751, 39908, 39779, 39863, 39746, 39768, 39892, 39857, 39889, 39862, 39923, 39836, 39803, 39770, 39852, 39781, 39813, 39871, 39841, 39802, 39920, 39751, 39918, 39780, 39745, 39915, 40870, 40894, 40934, 40912, 40846, 40868, 40849, 40904, 40909, 40928, 40921, 40930, 40848, 40887, 40897, 40929, 40860, 40916, 40886, 40882, 40893, 40875, 40880, 40850, 40925, 40945, 41080, 40961, 40964, 41120, 41131, 41039, 41051, 41138, 40966, 40952, 41102, 41003, 41054, 41045, 41078, 41011, 41104, 41116, 41041, 40969, 41067, 41097, 41114, 41020, 47652, 47533, 47541, 47673, 47507, 47486, 47638, 47558, 47627, 47456, 47595, 47510, 47498, 47502, 47599, 47484, 47572, 47611, 47602, 47505, 47590, 47589, 47639, 47647, 47493, 50272, 50350, 50326, 50258, 50279, 50309, 50427, 50378, 50292, 50259, 50348, 50355, 50385, 50308, 50341, 50382, 50334, 50373, 50325, 50278, 50409, 50417, 50262, 50426, 50359, 19982, 19976, 20004, 19921, 19858, 19953, 19893, 19926, 19947, 19901, 19984, 19999, 19985, 19952, 20007, 19937, 19881, 19865, 19835, 19912, 19973, 19948, 19890, 19916, 19918, 44248, 44356, 44302, 44294, 44345, 44331, 44280, 44357, 44722, 44511, 44600, 44317, 44589, 44298, 44244, 44308, 44343, 44909, 44285, 44711, 44292, 44500, 44262, 44273, 44655, 23699, 23645, 23483, 23579, 23710, 23624, 23724, 23523, 23610, 23732, 23690, 23628, 23703, 23534, 23435, 23706, 23566, 23568, 23720, 23684, 23604, 23455, 23603, 23485, 23707, 54982, 54901, 54913, 54906, 54851, 54828, 54829, 55000, 54951, 54987, 54947, 54949, 54955, 54918, 54856, 54935, 54817, 54892, 54816, 54838, 54831, 54917, 54985, 55004, 54952, 20762, 20795, 20724, 20756, 20790, 20721, 20699, 20850, 20860, 20742, 20686, 20792, 20863, 20700, 20738, 20703, 20789, 20879, 20866, 20770, 20869, 20874, 20858, 20778, 20716, 22642, 22058, 22330, 22649, 22646, 22082, 22640, 22274, 22016, 22067, 22015, 22068, 22076, 22022, 22627, 22196, 22285, 22218, 22085, 22152, 22207, 22429, 22363, 22653, 22115, 55645, 55653, 55631, 55655, 55664, 55669, 55627, 55657, 55630, 55647, 55652, 55659, 55661, 55634, 55624, 55626, 55667, 55642, 55663, 55625, 55635, 55648, 55665, 55658, 55641, 55884, 55892, 55782, 55899, 55893, 55709, 55764, 55699, 55901, 55859, 55783, 55704, 55873, 55862, 55748, 55705, 55755, 55674, 55767, 55820, 55902, 55839, 55692, 55750, 55896, 56696, 56745, 56707, 56557, 56580, 56656, 56627, 56655, 56714, 56608, 56693, 56670, 56748, 56692, 56625, 56675, 56642, 56597, 56561, 56702, 56687, 56673, 56605, 56728, 56571, 58535, 58492, 58528, 58558, 58450, 58592, 58463, 58569, 58470, 58476, 58523, 58546, 58480, 58447, 58506, 58497, 58475, 58474, 58575, 58510, 58520, 58618, 58504, 58478, 58577, 18324, 19159, 10785, 12869, 16735, 10364, 10359, 18412, 10347, 8883, 18406, 19026, 14079, 13409, 10498, 18959, 8857, 9014, 9248, 12721, 12862, 18393, 9409, 12717, 8830, 17709, 17658, 17682, 17680, 17646, 17755, 18167, 18201, 17720, 17655, 17690, 18156, 18303, 17713, 17707, 17879, 18045, 17747, 18001, 18280, 17714, 17708, 17660, 17669, 17665, 19792, 19350, 19816, 19552, 19652, 19441, 19826, 19789, 19810, 19306, 19797, 19267, 19812, 19619, 19279, 19260, 19258, 19269, 19290, 19363, 19719, 19831, 19300, 19264, 19302, 23292, 23419, 22774, 22730, 23409, 23004, 22691, 22993, 22715, 23303, 22717, 23337, 22751, 22702, 23215, 22704, 22662, 23370, 22837, 22735, 22772, 23429, 22737, 22709, 22763, 51809, 51876, 51853, 51829, 51870, 51807, 51812, 51878, 51863, 51814, 51805, 51825, 51854, 51822, 51813, 51835, 51818, 51808, 51827, 51881, 51856, 51802, 51880, 51819, 51850, 52086, 52068, 51983, 52130, 51982, 52084, 52059, 51903, 51899, 52150, 52058, 52038, 51939, 52096, 52103, 52137, 52032, 51896, 51993, 51925, 51953, 52007, 52034, 51980, 52071, 50813, 50815, 50825, 50857, 50957, 51068, 51128, 50790, 51020, 50858, 50820, 50954, 51112, 50929, 50870, 51078, 51246, 51024, 50913, 51228, 51192, 51179, 50967, 51086, 50760]
import json
import os
fi_train = '/ps/project/irotate/vit_data/ap-10k/annotations/ap10k-train-split1.json'
fm = '/ps/project/irotate/vit_data/s_ap10_zebras/train/train_full.json'
fo = '/ps/project/irotate/vit_data/s_ap10_zebras/train/train_1250.json'

base_file_train = json.load(open(fi_train,'r'))

In [99]:
fo_train = '/ps/project/irotate/vit_data/ap-10k/annotations/ap10k-train-split1_1250.json'

fnt_tokeep = []
for idx, ann in enumerate(base_file_train['annotations']):
    if ann['image_id'] in anno_list:
        fnt_tokeep.append(idx)
for i in fnt_tokeep[::-1]:
    base_file_train['annotations'].pop(i)

json.dump(base_file_train, open(fo_train,'w'))

In [ ]:
fn_tokeep = []
for i in base_file_train['images']:
    if i['id'] in anno_list:
        fn_tokeep.append(i['file_name'])

In [ ]:
input_file = json.load(open(fm,'r'))

m_mix = {}
cnt = 0
for i in input_file['images']:
    if i['file_name'] in fn_tokeep or not i['file_name'].startswith("000"):
        m_mix[i['id']] = i['file_name']
    if not i['file_name'].startswith("000"):
        cnt += 1

In [ ]:
cnt = []
for i in input_file['annotations']:
    if i['image_id'] in m_mix and not m_mix[i['image_id']].startswith("000"):
        cnt.append(i['category_id'])
print(len(cnt))

In [ ]:
to_pop = []
for idx, ann in enumerate(input_file['annotations']):
    if not ann['image_id'] in m_mix:
        to_pop.append(idx)

In [ ]:
for i in to_pop[::-1]:
    input_file['annotations'].pop(i)
    
print(len(input_file['annotations']))

json.dump(input_file, open(fo,'w'))

# Create X images for validation in ap10k

In [ ]:
import json
import os
fi_valid = '/ps/project/irotate/vit_data/ap-10k/annotations/ap10k-val-split1.json'

valid_targets = json.load(open(fi_valid,'r'))
bucket_cats = {}
for ann in valid_targets['annotations']:
    if ann['category_id'] not in bucket_cats:
        bucket_cats[ann['category_id']] = set()
    bucket_cats[ann['category_id']].add(ann['image_id'])
      
import random
for i in bucket_cats:
    if len(bucket_cats[i]) > 6:
        bucket_cats[i] = random.sample(bucket_cats[i], k=6)
print(sum([len(bucket_cats[i]) for i in bucket_cats]))

In [ ]:
fm = '/ps/project/irotate/vit_data/s_ap10_zebras/valid/valid_full_3k.json'
fo = '/ps/project/irotate/vit_data/s_ap10_zebras/valid/valid_1250_3k.json'

In [ ]:
ids_to_keep = []
for i in bucket_cats:
    ids_to_keep.extend(list(bucket_cats[i]))

fn_tokeep = []
for i in valid_targets['images']:
    if i['id'] in ids_to_keep:
        fn_tokeep.append(i['file_name'])

In [ ]:
input_file = json.load(open(fm,'r'))

m_mix = {}
for i in input_file['images']:
    if i['file_name'] in fn_tokeep or not i['file_name'].startswith("000"):
        m_mix[i['id']] = i['file_name']

In [ ]:
to_pop = []
for idx, ann in enumerate(input_file['annotations']):
    if not ann['image_id'] in m_mix:
        to_pop.append(idx)
for i in to_pop[::-1]:
    input_file['annotations'].pop(i)

print(len(input_file['annotations']))

In [ ]:
cnt = 0
cnt_bis = 0
for i in input_file['annotations']:
    if m_mix[i['image_id']].startswith("000"):
        cnt += 1
    else:
        cnt_bis += 1
print(cnt)
print(cnt_bis)

In [ ]:
json.dump(input_file, open(fo,'w'))

In [115]:
fo_val = '/ps/project/irotate/vit_data/ap-10k/annotations/ap10k-val-split1_1250.json'
fi_val = '/ps/project/irotate/vit_data/ap-10k/annotations/ap10k-val-split1.json'
val_1250_mix = '/ps/project/irotate/vit_data/s_ap10_zebras/valid/valid_1250.json'

base_file_val = json.load(open(val_1250_mix,'r'))

m = {}
for i in base_file_val['images']:
    if i['file_name'].startswith("000"):
        m[i['id']] = i['file_name']
    
ims = []
for i in base_file_val['annotations']:
    if i['image_id'] in m:
        ims.append(m[i['image_id']])

In [118]:
base_file_val = json.load(open(fi_val))

torem = []
for i in base_file_val['images']:
    if i['file_name'] not in ims:
        torem.append(i['id'])



In [120]:
len(base_file_val['images']) - len(torem)

296

In [121]:
to_pop = []
for idx, ann in enumerate(base_file_val['annotations']):
    if ann['image_id'] in torem:
        to_pop.append(idx)

for i in to_pop[::-1]:
    base_file_val['annotations'].pop(i)

print(len(base_file_val['annotations']))

389


In [122]:
json.dump(base_file_val, open(fo_val,'w'))

# 36 horses tigDog

In [73]:
fi = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_17.json'

ti = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_17_train.json'
train_fnames = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/train_fnames.npy'
vi = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_17_val.json'
valid_fnames = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/valid_fnames.npy'

train_names = np.load(train_fnames)[:,0]
valid_names = np.load(valid_fnames)[:,0]

jsonfile = json.load(open(fi,'r'))
ids_torem = []
for idx, img in enumerate(jsonfile['images']):
    if img['file_name'].split("crop_")[1] not in train_names:
        ids_torem.append(img['id'])

torem = []
for idx, ann in enumerate(jsonfile['annotations']):
    if ann['image_id'] in ids_torem:
        torem.append(idx)

for i in torem[::-1]:
    jsonfile['annotations'].pop(i)
json.dump(jsonfile, open(ti,'w'))

jsonfile = json.load(open(fi,'r'))
ids_torem = []
for idx, img in enumerate(jsonfile['images']):
    if img['file_name'].split("crop_")[1] not in valid_names:
        ids_torem.append(img['id'])

torem = []
for idx, ann in enumerate(jsonfile['annotations']):
    if ann['image_id'] in ids_torem:
        torem.append(idx)

for i in torem[::-1]:
    jsonfile['annotations'].pop(i)
json.dump(jsonfile, open(vi,'w'))

In [79]:
ti = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27_train.json'
vi = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27_val.json'
tr = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27_train_red.json'
vr = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27_val_red.json'

jsonfile = json.load(open(ti,'r'))
torem_train = random.sample(range(len(jsonfile['annotations'])), k=len(jsonfile['annotations'])-80)
torem_train.sort()
for i in torem_train[::-1]:
    jsonfile['annotations'].pop(i)
json.dump(jsonfile, open(tr,'w'))

jsonfile = json.load(open(vi,'r'))
torem_val = random.sample(range(len(jsonfile['annotations'])), k=len(jsonfile['annotations'])-19)
torem_val.sort()
for i in torem_val[::-1]:
    jsonfile['annotations'].pop(i)
json.dump(jsonfile, open(vr,'w'))

In [80]:
ti = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_17_train.json'
vi = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_17_val.json'
tr = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_17_train_red.json'
vr = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_17_val_red.json'
jsonfile = json.load(open(ti,'r'))
for i in torem_train[::-1]:
    jsonfile['annotations'].pop(i)
json.dump(jsonfile, open(tr,'w'))

jsonfile = json.load(open(vi,'r'))
for i in torem_val[::-1]:
    jsonfile['annotations'].pop(i)
json.dump(jsonfile, open(vr,'w'))

In [ ]:
jsonfile['categories']


In [ ]:
# get full train and validations set with an 80/20 split

In [ ]:
import json
import os
import numpy as np
ti = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27_train.json'
vi = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27_val.json'

tf_17 = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_17_train_full.json'
vf_17 = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_17_val_full.json'

fi = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27.json'

In [ ]:
jsonfile = json.load(open(fi,'r'))
tra_ims = []
val_ims = []

json_ti = json.load(open(ti,'r'))
json_vi = json.load(open(vi,'r'))

subset_ti = []
subset_ti_images = []
subset_vi = []
subset_vi_images = []

for i in json_ti['annotations']:
    subset_ti.append(i['id'])
    subset_ti_images.append(i['image_id'])

for i in json_vi['annotations']:
    subset_vi.append(i['id'])
    subset_vi_images.append(i['image_id'])
    
cnt_ti = 0
cnt_vi = 0

for i in jsonfile['images']:
    if i['id'] in subset_ti_images:
        tra_ims.append(i['id'])
    elif i['id'] in subset_vi_images:
        val_ims.append(i['id'])
    else:
        if np.random.rand() < 0.8:
            tra_ims.append(i['id'])
        else:
            val_ims.append(i['id'])


In [ ]:
jsonfile = json.load(open(fi,'r'))

torem = []
for idx, ann in enumerate(jsonfile['annotations']):
    if ann['image_id'] not in tra_ims:
        torem.append(idx)

for i in torem[::-1]:
    jsonfile['annotations'].pop(i)

json.dump(jsonfile, open(tf_17,'w'))

jsonfile = json.load(open(fi,'r'))

torem = []
for idx, ann in enumerate(jsonfile['annotations']):
    if ann['image_id'] not in val_ims:
        torem.append(idx)
for i in torem[::-1]:
    jsonfile['annotations'].pop(i)
json.dump(jsonfile, open(vf_17,'w'))

In [ ]:
# check
jsonfile = json.load(open(tf_17,'r'))
tr_im_names = {}
for i in jsonfile['images']:
    tr_im_names[i['id']] = i['file_name']
    
jsonfile = json.load(open(vf_17,'r'))
val_im_names = {}
for i in jsonfile['images']:
    val_im_names[i['id']] = i['file_name']



In [45]:
full_UDA = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27.json'
full_train = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_17_train_full.json'
full_train_27 = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27_train_full.json'
full_val = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_17_val_full.json'
full_val_27 = '/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27_val_full.json'

nval_ann = 0
ntrain_ann = 0
jsonfile = json.load(open(full_train,'r'))
ntrain_ann = len(jsonfile['annotations'])
jsonfile = json.load(open(full_val,'r'))
nval_ann = len(jsonfile['annotations'])


In [46]:
jsonfile = json.load(open(full_UDA,'r'))
torem = []
for idx, ann in enumerate(jsonfile['annotations']):
    if ann['image_id'] not in tra_ims:
        torem.append(idx)
if len(jsonfile['annotations']) - len(torem)== ntrain_ann:
    for i in torem[::-1]:
        jsonfile['annotations'].pop(i)
    json.dump(jsonfile, open(full_train_27,'w'))


In [47]:
jsonfile = json.load(open(full_UDA,'r'))
torem = []
for idx, ann in enumerate(jsonfile['annotations']):
    if ann['image_id'] not in val_ims:
        torem.append(idx)
if len(jsonfile['annotations']) - len(torem)== nval_ann:
    for i in torem[::-1]:
        jsonfile['annotations'].pop(i)
    json.dump(jsonfile, open(full_val_27,'w'))

In [58]:
tmp = np.load('/media/ebonetto/WindowsData/train_idxs_by_video.npy')

In [61]:
len(tmp)

8380

In [62]:
jsonfile = json.load(open('/media/ebonetto/WindowsData/pose_zebras_sw/all_about_zebras/ViTPose/data/unsup_dom_adapt/real_horse_crop_27.json','r'))

In [63]:
jsonfile['images'][0]

{'id': 0, 'file_name': 'crop_00018004.jpg', 'width': 256, 'height': 256}